<h1>Method of aliases</h1>
<p>This is a summary of a the alias sampling method drawn from wikipedia, the <a href="https://lips.cs.princeton.edu/">LIPS blog</a> and other sources. 
THe alias method is a family of efficient algorithms for sampling from a discrete probability distribution, 
due to A. J. Walker.[1][2] That is, it returns integer values 1 &leq; i &leq; n according to some arbitrary probability 
distribution pi. The algorithms typically use O(n log n) or O(n) preprocessing time, after which random values can be drawn from the distribution in O(1) time.</p>
<p>The starting point is a finite discrete distribution. $$P(X=v_i) = p_i$$</p>
<p>The method constructs a series of numbers $P[j]\in [0,1], j=1,\ldots,n$ and of aliases $Y[j], j=1,\ldots,n$ and uses them to return a random value with the distribution $P(X=v_i) = p_i$.

<p>The basic idea is to observe that any discrete distribution over $K$ outcomes can be turned into a uniform 
distribution over (possibly degenerate) binary outcomes. Since sampling from a uniform distribution can be done 
in constant time, it is easy to sample once you've computed an appropriate mixture. The setup cost is linear 
in $K$. You can convince yourself that such a mixture exists using induction. </p>
<p>First, if $K=1$, it is clearly easy.
For $K>1$, find $k_{\sf min} = \arg\min_k \pi_k$ and $k_{\sf max} = \arg\max_k \pi_k$. We know that 
$\pi_{k_{\sf min}}\leq 1/K$, so use these two to create a binary mixture between outcomes 
$k_{\sf min}$ and $k_{\sf max}$ where this component now owns all of the probability mass for
$k_{\sf min}$ but only $1/K - \pi_{k_{\sf min}}$ of the mass for $k_{\sf max}$. Having done this, we now 
have a new discrete distribution with $K-1$ outcomes, which we can iterate until there is only one outcome.
</p>

<h2>Alias setup: Table generation</h2>
<p>Say we have $K$ discrete items. </p>

In [ ]:
import numpy        as np
import numpy.random as npr

def alias_setup(probs):
    K         = len(probs)
    q = np.zeros(K) # array of alias-normalized probabilities
    Alias     = np.zeros(K, dtype=np.int) # J

<p>To generate the table, first initialize $U_i = Kp_i$. While doing this, divide the table entries into three categories:</p>
<ul>
    <li>The “overfull” group, where $U_i > 1$ (<tt>larger</tt> in the following code)</li>
    <li>The “underfull” group, where $U_i \leq 1$  (<tt>smaller</tt> in the following code)and
 </ul>
 <p>The following code divides all entries into the groups smaller and larger (Note that if one of these exists, the other must, as well).</p>

In [3]:
def alias_setup(probs):
    # (...)

    # Sort the data into the outcomes with probabilities
    # that are larger and smaller than 1/K.
    smaller = []
    larger  = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

<p>In the following, we arbitrarily choose a too-big and a too-small entry. Note that if we begin with an odd number
of entries, then the last time we do a pop, the entry should be 1. We then allocate the unused space in entry $j$ to outcome $i$, 
by setting $K_j = i$ (<tt>J[small] = large</tt>).</p>
<p>Remove the allocated space from entry $i$ by changing $U_i = U_i − (1 − U_j) = U_i + U_j − 1$. Entry $j$ is now exactly full. </p>
<p>There are two options how to "rearrange" the probabilities. See this <a href="http://www.keithschwarz.com/darts-dice-coins/">webpage</a> for intuition.</p>

In [ ]:
def alias_setup(probs):
    # (...)
    # (...)
    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        Alias[small] = large
        q[large] = q[large] - (1.0 - q[small])

        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return Alias, q

<h2>Alias draw</h2>
<p>To draw, we perform the following steps</p>
<ul>
    <li>Generate a fair die roll from an $n$-sided die; call the side $i$</li>
    <li>Flip a biased coin that comes up heads with probability $Prob[i]$</li>
    <li>If the coin comes up "heads," return $i$</li>
    <li>Otherwise, return Alias[i]</li>
</ul>
.

In [ ]:
def alias_draw(J, q):
    K  = len(J)

    # Draw from the overall uniform mixture.
    kk = int(np.floor(npr.rand()*K))

    # Draw from the binary mixture, either keeping the
    # small one, or choosing the associated larger one.
    if npr.rand() < q[kk]:
        return kk
    else:
        return J[kk]